In [3]:
import pandas as pd
from datetime import datetime
import plotly.express as px
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
from yahooquery import Ticker

In [1]:
from prophet import Prophet
import pandas_ta as ta
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

In [13]:
# Name by which it is represented on the stock exchange.
symbol = "AMER3.SA"

# Query stock in yahoo finance
stock = Ticker(symbol)

# Get 48 months data for enough sample terms
history = stock.history(period="all")

# Let only date as index
history.reset_index(level=["symbol"], inplace=True)

# Create date column
history['date'] = history.index

# Reindex data using a DatetimeIndex
history.set_index(pd.DatetimeIndex(history.index), inplace=True)

# select features that have interest to us
data = history[['date','adjclose']].copy()

,symbol,open,high,low,close,volume,adjclose,date
date,,,,,,,,
2023-02-09 12:30:39-03:00,AMER3.SA,1.02,1.17,0.96,1.03,81051600,1.03,2023-02-09 12:30:39-03:00


In [16]:
# use technical analyses using 21 one days and append to our dataset
data.ta.ema(close='adjclose', length=21, append=True)

# Drop empty values
data.dropna(inplace=True)

AttributeError: 'DataFrame' object has no attribute 'ta'

In [21]:
import yfinance as yf
ticker = "AMER3.SA"
data = yf.download(ticker, start='2022-02-02', end='2023-02-02').reset_index()

[*********************100%***********************]  1 of 1 completed


In [22]:
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-02-02,31.330000,32.439999,30.580000,31.209999,31.209999,7300100
1,2022-02-03,31.360001,32.369999,30.670000,32.000000,32.000000,6719900
2,2022-02-04,32.160000,32.209999,30.530001,32.150002,32.150002,7005100
3,2022-02-07,32.299999,32.759998,31.730000,32.200001,32.200001,3894500
4,2022-02-08,32.070000,33.529999,31.120001,33.270000,33.270000,6669900
...,...,...,...,...,...,...,...
246,2023-01-26,0.950000,1.050000,0.940000,1.030000,1.030000,86642800
247,2023-01-27,1.050000,1.200000,1.040000,1.200000,1.200000,98652200
248,2023-01-30,1.310000,1.640000,1.280000,1.450000,1.450000,204452600
249,2023-01-31,1.610000,1.750000,1.580000,1.750000,1.750000,212962100


In [27]:
#Plot
fig = px.line(data, x='Date', y='Adj Close')

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1D", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="Ano", step="year", stepmode="todate"),
            dict(count=1, label="1a", step="year", stepmode="backward"),
            dict(step="all", label="Máx")
        ])
    )
)

#['month', 'year', 'day', 'hour', 'minute', 'second',  'all']

In [44]:
import plotly.graph_objects as go
import pandas as pd


df = data

fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open'], high=df['High'],
                low=df['Low'], close=df['Close'])
                      ])

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1D", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="Ano", step="year", stepmode="todate"),
            dict(count=1, label="1a", step="year", stepmode="backward"),
            dict(step="all", label="Máx")
        ])
    )
)

fig.update_layout(
    title='The Great Recession',
    yaxis_title='AAPL Stock',
)

fig.update(layout_xaxis_rangeslider_visible=False)

fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# data
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')


# Create subplots and mention plot grid size
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
               vertical_spacing=0.03, subplot_titles=('OHLC', 'Volume'), 
               row_width=[0.2, 0.7])

# Plot OHLC on 1st row
fig.add_trace(go.Candlestick(x=df["Date"], open=df["AAPL.Open"], high=df["AAPL.High"],
                low=df["AAPL.Low"], close=df["AAPL.Close"], name="OHLC"), 
                row=1, col=1
)

# Bar trace for volumes on 2nd row without legend
fig.add_trace(go.Bar(x=df['Date'], y=df['AAPL.Volume'], showlegend=False), row=2, col=1)

# Do not show OHLC's rangeslider plot 
fig.update(layout_xaxis_rangeslider_visible=False)
fig.show()

In [90]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# data
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')
df = data



# Create subplots and mention plot grid size
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
               vertical_spacing=0.03, subplot_titles=('OHLC', 'Volume'), 
               row_width=[0.2, 0.7])

fig.update_xaxes(
    rangeslider_visible=True,
    
    rangeselector=dict(
    visible = True,
    x = 0, y = 1,
        buttons=list([
            dict(count=1, label="1D", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="Ano", step="year", stepmode="todate"),
            dict(count=1, label="1a", step="year", stepmode="backward"),
            dict(step="all", label="Máx")
        ])
    )
)

# Plot OHLC on 1st row
fig.add_trace(go.Candlestick(x=df["Date"], open=df["Open"], high=df["High"],
                low=df["Low"], close=df["Close"], name="OHLC"), 
                row=1, col=1
)

# Bar trace for volumes on 2nd row without legend
fig.add_trace(go.Bar(x=df['Date'], y=df['Volume'], showlegend=True), row=2, col=1)


# Do not show OHLC's rangeslider plot 
fig.update(layout_xaxis_rangeslider_visible=False)
fig.show()

In [97]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# data
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')
df = data



# Create subplots and mention plot grid size
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
               vertical_spacing=0.03, subplot_titles=('OHLC', 'Volume'), 
               row_width=[0.2, 0.7])

fig.update_xaxes(
    rangeslider_visible=False,
    
    rangeselector=dict(
    visible = True,
    x = 0, y = 1,
    bgcolor = 'rgba(150, 200, 250, 0.4)',
    font = dict( size = 13 ),
    buttons=list([
        dict(count=1,
             label='reset',
             step='all'),
        dict(count=1,
             label='1yr',
             step='year',
             stepmode='backward'),
        dict(count=3,
            label='3 mo',
            step='month',
            stepmode='backward'),
        dict(count=1,
            label='1 mo',
            step='month',
            stepmode='backward'),
        dict(step='all')
    ]))
)

# Plot OHLC on 1st row
fig.add_trace(go.Candlestick(x=df["Date"], open=df["Open"], high=df["High"],
                low=df["Low"], close=df["Close"], name="OHLC"), 
                row=1, col=1
)

# Bar trace for volumes on 2nd row without legend
fig.add_trace(go.Bar(x=df['Date'], y=df['Volume'], showlegend=False), row=2, col=1)


# Do not show OHLC's rangeslider plot 
fig.update(layout_xaxis_rangeslider_visible=False)
fig.show()

In [99]:
import yfinance as yf
ticker = "AMER3.SA"
data = yf.download(ticker, start='2022-02-02', end='2023-02-02').reset_index()

[*********************100%***********************]  1 of 1 completed


In [100]:
import plotly.express as px
import plotly.offline as pyo

df = yf.download(ticker, start='2022-02-02', end='2023-02-02')

fig = px.candlestick(df)
pyo.iplot(fig)

[*********************100%***********************]  1 of 1 completed


AttributeError: module 'plotly.express' has no attribute 'candlestick'